<a href="https://colab.research.google.com/github/BroDeb/BroDeb/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import sys
sys.path.insert(1,r'/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/2019 1111 Final_codes_and_data/Step_5_SVMOnDifferentCVs')
!pip install xlsxwriter

date = datetime.datetime.now()
day = str(date.day)
month = str(date.month)
year = str(date.year)
if len(day) <= 1:
    day = '0' + day
if len(month) <= 1:
    month = '0' + month
    
date = year+month+day



root_path = r'/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG'

#VTVF3YrsFiles
vtvf3yr_split_path = r'/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 2/2019 1108 VTVF Split.xlsx'
vtvf3yr_extracted_features_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 3 tsfresh/ECGTsfreshExtractedFeaturesVtvf3YRLabel20211126.csv'
vtvf3yr_y_labels_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 3 tsfresh/TsfreshVtvf3yr_y_series20211125.csv'
vtvf3yr_features_coefficients = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 4 reg/FeaturesWeights0.05L1Vtvf3yrs20211126.csv'
#
death3yr_split_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 2/2019 1108 death 3 year split.xlsx'
death3yr_extracted_features_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 3 tsfresh/ECGTsfreshExtractedFeaturesDeath3YRLabel20211126.csv'
death3yr_y_labels_path ='/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 3 tsfresh/TsfreshDeath3yr_y_series20211125.csv'
death3yr_features_coefficients = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/Step 4 reg/FeaturesWeights0.05L1Death3yrs20211126.csv'


number_of_features = 101
regularization_type = 'L1'


for i in range(2):
    SVM_results = pd.DataFrame()
    SVM_results_Train = pd.DataFrame()
    SVM_results_Val = pd.DataFrame()
    if i == 0:
        Split_PD = pd.ExcelFile(vtvf3yr_split_path) 
        extracted_features = pd.read_csv(vtvf3yr_extracted_features_path, index_col = 'id')
        Labels = pd.read_csv(vtvf3yr_y_labels_path, index_col = 'id')
        features_coeff = pd.read_csv(vtvf3yr_features_coefficients, index_col = 'Unnamed: 0')
        output_label = 'VT_VF in 3Y'
        writer = pd.ExcelWriter('SVCLinear_'+date+'_3SD_singleBeat_'+str(number_of_features)+'features_'+'_'+regularization_type+'_'+output_label+'.xlsx', engine='xlsxwriter')

    elif i ==1:
        Split_PD = pd.ExcelFile(death3yr_split_path) 
        extracted_features = pd.read_csv(death3yr_extracted_features_path, index_col = 'id')
        Labels = pd.read_csv(death3yr_y_labels_path, index_col = 'id')
        output_label = 'Death in 3Y'
        features_coeff = pd.read_csv(death3yr_features_coefficients, index_col = 'Unnamed: 0')
        writer = pd.ExcelWriter('SVCLinear_'+date+'_3SD_singleBeat_'+str(number_of_features)+'features_'+'_'+regularization_type+'_'+output_label+'.xlsx', engine='xlsxwriter')

    features_coeff['abs feature coefficient'] = features_coeff['Feature Coefficient at C = 0.05'].abs()
    features_coeff.sort_values('abs feature coefficient', ascending = False, inplace = True, axis = 0)
    topXFeaturesNames = features_coeff['Feature Name'][:number_of_features]
    extracted_features = extracted_features[topXFeaturesNames]
    
    for sheet in ['CV1','CV2','CV3','CV4','CV5','CV6','CV7','CV8','CV9','CV10']:
        if True:
            sheet_name = sheet
            outcome_sheet = pd.read_excel(Split_PD, sheet_name = sheet)
            
            Train_dict = {}
            Val_dict = {}
            
            for index, row in outcome_sheet.iterrows():
                if index >= 0:     
                    split_num = row['split_num']
                    if split_num == 0:
                        Train_dict[str(row['patient'])] = 1
                    elif split_num == 1:
                        Val_dict[str(row['patient'])] = 3
                        
            Train_pd = pd.DataFrame()
            Train_Y = pd.DataFrame()
            Val_pd = pd.DataFrame()
            Val_Y = pd.DataFrame()

        
            count = 0
            for index, row in extracted_features.iterrows():
                MAP_features = extracted_features.iloc[[count]]
                MAP_label = Labels.iloc[[count]]
                patient_id = str(index)[:5]
                if patient_id in Train_dict.keys():
                    Train_pd = Train_pd.append(MAP_features)
                    Train_Y = Train_Y.append(MAP_label)
                elif patient_id in Val_dict.keys():
                    Val_pd = Val_pd.append(MAP_features)
                    Val_Y = Val_Y.append(MAP_label)
                count += 1
#                if count >5707:
#                    break
        
            from SupportVectorMachine import svc_model
            C_s = [1]# [1e-3, 1e-2, 0.5e-1, 1e-1, 1,10,100]
            svm_kernal = 'linear'
            for i,C in enumerate(C_s):
                svc_model = svc_model(Train_pd,Train_Y['0'], C=C)

                y_pred_Train_top_X_Features = svc_model.predict(Train_pd)
                y_pred_Val_top_X_Features = svc_model.predict(Val_pd)
    #            y_pred_Test_30Features = svc_30Features.predict(Test_pd_top30)
            
                from calc_metrics_v2 import calc_classification_metrics
                
                Train_metrics_top_X_Features = calc_classification_metrics(Train_Y['0'],y_pred_Train_top_X_Features)
                Val_met_top_X_Features = calc_classification_metrics(Val_Y['0'],y_pred_Val_top_X_Features)
        #        Test_met_30Features = calc_classification_metrics(Test_Y.values,y_pred_Test_30Features)
                SVM_results_Train = SVM_results_Train.append({'num of top features': number_of_features, 'Cross Validation': sheet, 'split': 'Train','C (SVM param)': C, 
                                                  'Accuracy':Train_metrics_top_X_Features[1], 'Sensitivity':Train_metrics_top_X_Features[2], 
                                                  'Specificity':Train_metrics_top_X_Features[3], 'PPV':Train_metrics_top_X_Features[4], 
                                                  'NPV':Train_metrics_top_X_Features[5]}, ignore_index=True)
                SVM_results_Val = SVM_results_Val.append({'num of top features': number_of_features, 'Cross Validation': sheet, 'split': 'Val','C (SVM param)': C, 
                                                  'Accuracy':Val_met_top_X_Features[1], 'Sensitivity':Val_met_top_X_Features[2], 
                                                  'Specificity':Val_met_top_X_Features[3], 'PPV':Val_met_top_X_Features[4], 
                                                  'NPV':Val_met_top_X_Features[5]}, ignore_index=True)
                
#                Val_results = Val_Y.to_frame()
                Val_results = Val_Y.rename(columns={0: "True Labels"})
                Val_results['Predicted Labels'] = y_pred_Val_top_X_Features
                
#                Train_results = Train_Y.to_frame()
                Train_results = Train_Y.rename(columns={0: "True Labels"})
                Train_results['Predicted Labels'] = y_pred_Train_top_X_Features
                
                Val_results.to_excel(writer, sheet_name='Validation single labels '+ sheet)
                Train_results.to_excel(writer, sheet_name='Training single labels '+ sheet)
                
    
    # Write each dataframe to a different worksheet.
    SVM_results_Val.to_excel(writer, sheet_name='Results_Val')
    SVM_results_Train.to_excel(writer, sheet_name='Results_Train')
    
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

     |████████████████████████████████| 149 kB 15.5 MB/s 
